# Training: GDrive x BigQuery x GSheets
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/open?id=19XZJFzkBj9QuLAfyRMbXj1-v7d9mAtYq)

This notebook provides guidance on how you can connect your [Google Colab](https://colab.research.google.com/notebooks/welcome.ipynb#) instance to the following:

- Google Drive (GDrive) 
- BigQuery (BQ)
- Google Sheets (GSheets)

The interaction of these three is useful if you want to extract data from BigQuery and put it into a Google Sheet.

***

## Contents

The outline of this notebook is:

1. Mount GDrive
1. Connect to BigQuery
    + Import data from there
1. Prepare your data
1. Open and save into GSheets

***

## Set-up
We need to install and load the necessary packages and authenticate ourselves to use things like GDrive, BQ and GSheets.



In [0]:
# load packages
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

import pandas as pd
import re

import gspread
import gspread_dataframe as gd

***

## GDrive
We need to mount our GDrive to this session so we can access it. This location will be where we store the prepared data for other to use, a shared drive.

In [4]:
# mount GDrive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# show files in current directory
!ls

drive  sample_data


In [6]:
# navigate to folder of interest
%cd "drive/My Drive/Projects/training_googlecolaboratory"

/content/drive/My Drive/Projects/training_googlecolaboratory


***

## BigQuery
We connect to BigQuery to import data into our Google Colab session.

> *Note: Colab cannot take too much data so do not import too much!*

In [0]:
# pretty table display
%load_ext google.colab.data_table

In [0]:
# authenticate to access Google Cloud SDK to use BQ 
auth.authenticate_user()

In [0]:
# save output in a variable, `data_hn_stories`
%%bigquery --project training-265812 data_hn_stories

SELECT 
  `id`
  ,`by` AS username_submitter
  ,`score`
  ,`time_ts`
  ,`title`
  ,`url`
  ,`text`
  ,`deleted`
  ,`dead`
  ,`descendants`
  ,`author` AS username_author
FROM `bigquery-public-data.hacker_news.stories` 
WHERE title IS NOT NULL
  AND title NOT IN  ('', 'Placeholder')
  AND text IS NOT NULL
  AND text NOT IN ('')

In [15]:
# view data
data_hn_stories.info()
data_hn_stories

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218621 entries, 0 to 218620
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   id                  218621 non-null  int64              
 1   username_submitter  218621 non-null  object             
 2   score               218621 non-null  int64              
 3   time_ts             218621 non-null  datetime64[ns, UTC]
 4   title               218621 non-null  object             
 5   url                 217250 non-null  object             
 6   text                218621 non-null  object             
 7   deleted             0 non-null       object             
 8   dead                95795 non-null   object             
 9   descendants         210069 non-null  float64            
 10  username_author     218621 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(7)
memory usage: 18.3+

,id,username_submitter,score,time_ts,title,url,text,deleted,dead,descendants,username_author
0,6988445,cflick,0,2013-12-31 01:55:02+00:00,Appshare,http://chadflick.ws/appshare.html,Did facebook or angrybirds pay you? We will!,None,True,NaN,cflick
1,2130263,alikamp,0,2011-01-22 10:41:13+00:00,A Handfull of Gold.,,A handful of gold. Im sure we all believe that...,None,True,NaN,alikamp
2,7410550,jeassonlens,0,2014-03-16 18:28:18+00:00,Fastest Growing Skin Care Supplement for Incre...,http://naturosciences.com/,Naturo Sciences is a health &amp; beauty speci...,None,True,NaN,jeassonlens
3,7164302,annawright010,0,2014-02-02 01:08:27+00:00,R4 3ds sdhc,http://www.r4i3dscards.co.uk,R4i3dscards.co.uk is a reliable online store t...,None,True,NaN,annawright010
4,7791964,limpeseunomebvw,0,2014-05-23 23:38:08+00:00,Empréstimo Com Nome Sujo,http://www.emprestimopessoal-bvw.com.br/empres...,"limpe seu nome online e pela internet, para se...",None,True,NaN,limpeseunomebvw
...,...,...,...,...,...,...,...,...,...,...,...
218616,1769910,joshu,240,2010-10-07 23:18:43+00:00,"Ask HN: Tools of the trade, 2010 edition",,"When I first started delicious, we had to host...",None,None,92.0,joshu
218617,6653437,whoishiring,240,2013-11-01 13:18:22+00:00,Ask HN: Who is hiring? (November 2013),,Please lead with the location of the position ...,None,None,391.0,whoishiring
218618,3408407,jpadvo,245,2011-12-30 18:00:34+00:00,"Spamhaus Nightmare: Domain Shut Down, No Notic...",,"I am writing this here on HN, because at the m...",None,None,95.0,jpadvo
218619,9439286,temp_-_,247,2015-04-25 19:14:13+00:00,Ask HN: Those who quit their jobs to travel th...,,I seem to read so many comments on discussion ...,None,None,168.0,temp_-_
